# Qualtrics integration and pipeline
Demonstration of how we will generate archetypes from the *Qualtrics* responses both for the current car as well as for each vintage (i.e. archetype defined by `vehicle_type` and `fuel_type`)

## Qualtrics

In [1]:
import os

from QualtricsAPI.Setup import Credentials

from QualtricsAPI import Responses

In [2]:
TOKEN = os.environ.get('QUALTRICS_API_KEY')
DATA_CENTER = os.environ.get('QUALTRICS_BASE_URL').split('.')[0]

In [3]:
Credentials().qualtrics_api_credentials(token=TOKEN, data_center=DATA_CENTER)

In [4]:
r = Responses()
survey_id = 'SV_6u4YzY24OAfhWfQ'
df_pre_study = r.get_survey_responses('SV_6u4YzY24OAfhWfQ')

In [5]:
df_pre_study.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,pers_characs_worries,pers_characs_forgiving,pers_characs_lazy,pers_characs_open,pers_characs_nervous,pers_characs_efficient,pers_characs_reserved,pers_characs_considerate,pers_characs_imaginative,pers_characs_relaxed
0,Anfangsdatum,Enddatum,Antworttyp,IP-Adresse,Aktueller Stand,Dauer (in Sekunden),Beendet,Erfassungsdatum,Antwort-ID,Nachname des Empfängers,...,"Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg...","Bitte geben Sie jeweils an, wie stark die folg..."
1,"{""ImportId"":""startDate"",""timeZone"":""Z""}","{""ImportId"":""endDate"",""timeZone"":""Z""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""Z""}","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID168_7""}","{""ImportId"":""QID168_8""}","{""ImportId"":""QID168_9""}","{""ImportId"":""QID168_10""}","{""ImportId"":""QID168_12""}","{""ImportId"":""QID168_13""}","{""ImportId"":""QID168_14""}","{""ImportId"":""QID168_15""}","{""ImportId"":""QID168_16""}","{""ImportId"":""QID168_17""}"
2,2022-05-11 06:42:17,2022-05-11 06:59:46,0,194.165.44.169,100,1048,1,2022-05-11 06:59:48,R_3qe7fVb9Ma5yEyO,Bos,...,4,4,2,3,4,4,4,4,4,3
3,2022-05-11 06:05:01,2022-05-11 07:08:18,0,165.225.95.4,100,3797,1,2022-05-11 07:08:20,R_ZepQebZSlABxEsh,Bauer,...,2,3,2,4,2,4,3,4,4,2
4,2022-05-11 06:44:12,2022-05-11 07:16:31,0,31.10.155.126,100,1938,1,2022-05-11 07:16:32,R_2RacLJHZCDJ7J61,Bissig,...,3,2,1,4,3,4,2,4,4,3


## Workflow

In [1]:
import tcsscraper.scrape as tcs

from tcsscraper.helper import Car

In [7]:
Car.vehicle_classes

['Mikroklasse',
 'Kleinwagen',
 'Untere Mittelklasse',
 'Mittelklasse',
 'Obere Mittelklasse',
 'Luxusklasse',
 'Coupé / Sportwagen',
 'Cabriolet / Roadster',
 'SUV S',
 'SUV M',
 'SUV L',
 'SUV XL',
 'Minivan S',
 'Minivan M',
 'Minivan L']

In [8]:
Car.fuel_types

['Benzin',
 'Diesel',
 'Hybrid Benzin',
 'Hybrid Diesel',
 'Erdgas (CNG)',
 'Elektro',
 'Elektro mit Range Extender',
 'Plug-in Hybrid Benzin',
 'Plug-in Hybrid Diesel',
 'Wasserstoff / Elektro']

In [9]:
# car = Car('Luxusklasse', 'Elektro', 5)

In [4]:
car = Car('Luxusklasse', 'Benzin', 10)
print(car)

vehicle_type:     Luxusklasse
fuel_type:        Benzin
fuel_consumption: 10


In [11]:
similar_cars = tcs.get_similar_cars(vehicle_class=car.vehicle_class, fuel_type=car.fuel_type, fuel_consumption=car.fuel_consumption, km=20e3, canton='ZH', buffer=5, verbose=True)

TypeError: get_similar_cars() got an unexpected keyword argument 'vehicle_class'

In [ ]:
len(similar_cars)

17

In [8]:
def remove_unit(string):
    """Removes unit from string like 17'000 CHF/year -> float(17e3)

    Args:
        string (str): string with unit
    """
    number, unit = string.split(' ')
    number = number.replace("'", '')
    return float(number)

In [9]:
import pandas as pd

def generate_archetype(similar_cars):
    """Generates an archetypical car from a list of similar cars

    Args:
        similar_cars (list of dict): returned by get_similar_cars()
    """
    car_attributes = []
    for c in similar_cars:
        costs = c['costs']
        attrs = {
            'fix_cost': remove_unit(costs['Fixe Kosten']),
            'variable_cost': remove_unit(costs['Variable Kosten']),
            'cost_per_km': remove_unit(costs['Kilometerkosten'])
        }
        car_attributes.append(attrs)

    car_attributes = pd.DataFrame(car_attributes)

    archetype = dict(car_attributes.mean())

    return archetype
    

In [10]:
archetype = generate_archetype(similar_cars)
archetype

NameError: name 'similar_cars' is not defined

## Pipeline

In [11]:
## generate data.frame with variables to feed to get_similar_cars
df_qualt = pd.DataFrame({
    'vehicle_type': ['SUV S', 'Luxusklasse', 'Mittelklasse'],
    'fuel_type': ['Benzin', 'Benzin', 'Benzin'],
    'fuel_consumption': [3, 7, 2],
    'km': [15e3, 20e3, 22e3],
    'canton': ['VD', 'AI', 'ZG']
})

df_qualt.head()

,vehicle_type,fuel_type,fuel_consumption,km,canton
0,SUV S,Benzin,3,15000.0,VD
1,Luxusklasse,Benzin,7,20000.0,AI
2,Mittelklasse,Benzin,2,22000.0,ZG


In [12]:
def helper(row, max=5, verbose=True):
    car = Car(row['vehicle_type'], row['fuel_type'], row['fuel_consumption'])
    init = 1
    if verbose:
        print(car)
    while(init < max+1):
        try:
            sc = tcs.get_similar_cars(car, row['km'], row['canton'], buffer=init, verbose=verbose)
        except:
            init += 1
            continue
        return sc
    return None

In [13]:

df_qualt['similar_cars'] = df_qualt.apply(lambda row: helper(row), axis=1)

vehicle_type:     SUV S
fuel_type:        Benzin
fuel_consumption: 3
No similar cars found. A missmatch between model and consumption? Consider increasing buffer.
vehicle_type:     Luxusklasse
fuel_type:        Benzin
fuel_consumption: 7
No similar cars found. A missmatch between model and consumption? Consider increasing buffer.
vehicle_type:     Mittelklasse
fuel_type:        Benzin
fuel_consumption: 2
No similar cars found. A missmatch between model and consumption? Consider increasing buffer.


## TODO
- `fuel_consumption` with `fuel_type='electric'` does not work